# Data Transformations

In this notebook the IPL `ball-by-ball` csv is imported into a dataframe and some calculated fields are created. The IPL `matches` csv is used for removal of DL, no result, or tied games. For the calculated fields, the data is to be split into first inning and second inning. For the first inning dataframe there is a column for the team score after each over of the game and a column is also created with the wickets lost after each over with the over in question also stored in a column. This results in a row for each over of the match with in-game statistics across each of the columns. The second inning dataframe contains the over number, team total, team wickets lost, opposition score at the same stage in the first inning, opposition wickets lost at the same stage in the first inning, and the opposition total in the first inning. The predictive analysis will then determine whether a bat first win or bat second win will occur.

The Big Bash League (BBL) data as also transformed for further evaluation in the exact same process as the IPL data, as outlined above.

In [1]:
import pandas as pd
import collections

## IPL data

In [2]:
#import IPL data
deliveries = pd.read_csv('ball_by_ball.csv')
matches = pd.read_csv('matches.csv')

In [3]:
deliveries

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193463,1237181,1,12,5,RR Pant,SS Iyer,NM Coulter-Nile,0,0,0,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193464,1237181,1,12,6,RR Pant,SS Iyer,NM Coulter-Nile,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193465,1237181,1,13,1,RR Pant,SS Iyer,KH Pandya,0,1,1,0,0,NaN,NaN,NaN,wides,Delhi Capitals,Mumbai Indians
193466,1237181,1,13,2,RR Pant,SS Iyer,KH Pandya,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians


# No result, D/L affected game, and tied game removal

It was highlighted in the Data Analysis notebook that there were 4 no results, 19 D/L affected games and 13 tied games in the dataset that will be removed before any transformations are completed.
The ID's of these games are calculated and removed from the `deliveries` dataframe

In [4]:
# find no result matches by id and create an array with these id's
no_result = matches[matches.result.isnull()]
no_result.id.value_counts()
no = no_result.id.unique()
no

# find D/L affected matches by id and create an array with these id's
dl_result = matches[matches.method=='D/L']
dl_result.id.value_counts()
dl = dl_result.id.unique()
dl

# find tied matches by id and create an array with these id's
tie_result = matches[matches.result=='tie']
tie_result.id.value_counts()
tie = tie_result.id.unique()
tie

# edit the matches dataframe to remove D/L affected matches and tied matches
deliveries = deliveries[~deliveries.id.isin(no)]
deliveries = deliveries[~deliveries.id.isin(dl)]
deliveries = deliveries[~deliveries.id.isin(tie)]

In [5]:
deliveries

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193463,1237181,1,12,5,RR Pant,SS Iyer,NM Coulter-Nile,0,0,0,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193464,1237181,1,12,6,RR Pant,SS Iyer,NM Coulter-Nile,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians
193465,1237181,1,13,1,RR Pant,SS Iyer,KH Pandya,0,1,1,0,0,NaN,NaN,NaN,wides,Delhi Capitals,Mumbai Indians
193466,1237181,1,13,2,RR Pant,SS Iyer,KH Pandya,1,0,1,0,0,NaN,NaN,NaN,NaN,Delhi Capitals,Mumbai Indians


## BBL data

In [6]:
#import BBL data
bb_ball = pd.read_csv('bbl_deliveries.csv')
bb_match = pd.read_csv('bbl_matches.csv')

In [7]:
len(bb_ball.id.value_counts())
len(bb_match.id.value_counts())

414

In [8]:
bb_ball

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,524915,1,8,1,CA Lynn,ML Hayden,JR Hazlewood,0,0,0,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Sydney Sixers
1,524915,1,8,2,CA Lynn,ML Hayden,JR Hazlewood,2,0,2,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Sydney Sixers
2,524915,1,8,3,CA Lynn,ML Hayden,JR Hazlewood,0,0,0,0,1,caught,CA Lynn,MC Henriques,NaN,Brisbane Heat,Sydney Sixers
3,524915,1,8,4,ML Hayden,DT Christian,JR Hazlewood,1,0,1,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Sydney Sixers
4,524915,1,8,5,DT Christian,ML Hayden,JR Hazlewood,0,0,0,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Sydney Sixers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95818,1226886,2,0,3,CA Lynn,JL Denly,JA Richardson,0,0,0,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Perth Scorchers
95819,1226886,2,0,4,CA Lynn,JL Denly,JA Richardson,1,0,1,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Perth Scorchers
95820,1226886,2,0,5,JL Denly,CA Lynn,JA Richardson,2,0,2,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Perth Scorchers
95821,1226886,2,0,6,JL Denly,CA Lynn,JA Richardson,0,0,0,0,0,NaN,NaN,NaN,NaN,Brisbane Heat,Perth Scorchers


In [9]:
bb_ball.columns

Index(['id', 'inning', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'batsman_runs', 'extra_runs', 'total_runs', 'non_boundary', 'is_wicket',
       'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type',
       'batting_team', 'bowling_team'],
      dtype='object')

In [10]:
bb_match

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,524915,Sydney,2011-12-16,BJ Haddin,Sydney Cricket Ground,0,Sydney Sixers,Brisbane Heat,Brisbane Heat,bat,Sydney Sixers,wickets,7.0,N,NaN,BNJ Oxenford,PR Reiffel
1,524916,Melbourne,2011-12-17,DA Warner,Melbourne Cricket Ground,0,Melbourne Stars,Sydney Thunder,Sydney Thunder,field,Sydney Thunder,wickets,6.0,N,NaN,SD Fry,P Wilson
2,524917,Adelaide,2011-12-18,M Klinger,Adelaide Oval,0,Adelaide Strikers,Melbourne Renegades,Melbourne Renegades,field,Adelaide Strikers,runs,67.0,N,NaN,AJ Barrow,BNJ Oxenford
3,524918,Perth,2011-12-18,BW Hilfenhaus,Western Australia Cricket Association Ground,0,Perth Scorchers,Hobart Hurricanes,Hobart Hurricanes,bat,Hobart Hurricanes,runs,31.0,N,NaN,GA Abood,AP Ward
4,524919,Brisbane,2011-12-20,RJ Quiney,"Brisbane Cricket Ground, Woolloongabba",0,Brisbane Heat,Melbourne Stars,Melbourne Stars,bat,Melbourne Stars,runs,8.0,N,NaN,IH Lock,PR Reiffel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,1226882,Sydney,2021-02-06,JM Vince,Sydney Cricket Ground,0,Sydney Sixers,Perth Scorchers,Perth Scorchers,field,Sydney Sixers,runs,27.0,N,NaN,P Wilson,SJ Nogajski
410,1226883,Brisbane,2021-01-29,JJ Peirson,Brisbane Cricket Ground,0,Adelaide Strikers,Brisbane Heat,Adelaide Strikers,bat,Brisbane Heat,wickets,6.0,N,NaN,DM Koch,MW Graham-Smith
411,1226884,Canberra,2021-01-30,JM Vince,Manuka Oval,0,Perth Scorchers,Sydney Sixers,Perth Scorchers,bat,Sydney Sixers,wickets,9.0,N,NaN,P Wilson,SJ Nogajski
412,1226885,Canberra,2021-01-31,SD Heazlett,Manuka Oval,0,Sydney Thunder,Brisbane Heat,Brisbane Heat,field,Brisbane Heat,wickets,7.0,N,NaN,DM Koch,MW Graham-Smith


In [11]:
bb_match.result.value_counts()

wickets    205
runs       199
tie          4
Name: result, dtype: int64

In [12]:
# find no result matches by id and create an array with these id's
nr = bb_match[bb_match.result.isnull()]
nr.id.value_counts()
no = nr.id.unique()
no

# find D/L affected matches by id and create an array with these id's
dl_r = bb_match[bb_match.method=='D/L']
dl_r.id.value_counts()
dl = dl_r.id.unique()
dl

# find tied matches by id and create an array with these id's
tie_r = bb_match[bb_match.result=='tie']
tie_r.id.value_counts()
tie = tie_r.id.unique()
tie

# edit the matches dataframe to remove D/L affected matches and tied matches
bb_ball = bb_ball[~bb_ball.id.isin(no)]
bb_ball = bb_ball[~bb_ball.id.isin(dl)]
bb_ball = bb_ball[~bb_ball.id.isin(tie)]

In [13]:
len(bb_ball.id.value_counts())

382

The BBL data has been reduced to 382 completed games

In [14]:
ipl_ids = deliveries.id.unique()
bbl_ids = bb_ball.id.unique()

In [15]:
len(ipl_ids)

780

In [16]:
len(bbl_ids)

382

It can be seen that we have removed the no result, D/L affected, and tied games from both datasets and have 780 IPL games worth of ball_by_ball data and 382 BBL games worth of data

## Transformations


The next step is to create a function that allows for iteration through each of the rows within the dataframe while creating a summary statistic for each over of each game. We create a row for the end of each over and add this to our new df. This produces a dataset for the first innings of the IPL, the second innings of the IPL, the first innings of the BBL, and the second innings of the BBL.

In [17]:
COLUMN_NAMES = ['id', 'inning', 'over', 'runs', 'wickets']
#create a dataframe to hold this in game summary data
df_1 = pd.DataFrame(columns=COLUMN_NAMES)
COLUMN_NAMES = ['id', 'inning', 'over', 'runs', 'wickets', 'first_innings_runs', 'first_innings_wkts', 'first_innings_final_total']
#create a dataframe to hold this in game summary data
df_2 = pd.DataFrame(columns=COLUMN_NAMES)

COLUMN_NAMES = ['id', 'inning', 'over', 'runs', 'wickets']
df_3 = pd.DataFrame(columns=COLUMN_NAMES)
COLUMN_NAMES = ['id', 'inning', 'over', 'runs', 'wickets', 'first_innings_runs', 'first_innings_wkts', 'first_innings_final_total']
#create a dataframe to hold this in game summary data
df_4 = pd.DataFrame(columns=COLUMN_NAMES)

In [18]:
def home_bat_first(match, matches, match_id):
    # identify if the home team batted first
    inning1 = match[match['inning']==1]
    de = inning1.batting_team[inning1.id==match_id]
    ma = matches.team1[matches.id==match_id]
    if de.iloc[0]==str(ma.iloc[0]):
        return True
    else:
        return False

In [19]:
def get_over_summary(match):
    # create dictionary to hold overs values
    first_innings_overs = {}
    second_innings_overs = {}
    first_innings_wkts = {}
    second_innings_wkts = {}
    
    inning1 = match[match['inning']==1]
    for index, row in inning1.iterrows():
        # create over value if no data in overs, otherwise add to existing count for that over 
        if row['over'] not in first_innings_overs:
            first_innings_overs[row['over']] = row['total_runs']
        else:
            first_innings_overs[row['over']] += row['total_runs']
            
        # create wkts value if not already present
        if row['over'] not in first_innings_wkts:
            first_innings_wkts[row['over']] = row['is_wicket']
        else:
            first_innings_wkts[row['over']] += row['is_wicket']        
     
    # loop through innings 2
    inning2 = match[match['inning']==2]
    for index, row in inning2.iterrows():
        # create over value if no data in overs, otherwise add to existing count for that over 
        if row['over'] not in second_innings_overs:
            second_innings_overs[row['over']] = row['total_runs']
        else:
            second_innings_overs[row['over']] += row['total_runs']
            
        # create wkts value if not already present
        if row['over'] not in second_innings_wkts:
            second_innings_wkts[row['over']] = row['is_wicket']
        else:
            second_innings_wkts[row['over']] += row['is_wicket']

            
    return first_innings_overs, second_innings_overs, first_innings_wkts, second_innings_wkts
        

In [20]:
def get_cumulative_scores(first_d, second_d, first_wkts, second_wkts, home_bat_):
    scores = []
    if home_bat_:
        l = [first_d, second_d, first_wkts, second_wkts]
    #else:
     #   l = [second_d, first_d, second_wkts, first_wkts]
    i = 0
    
    for runs in first_d:
        total = 0
        for k, v in runs.items():
            total+=v
            scores.append(total)

    return scores

In [21]:
def get_match_summary(match_id, matches, deliveries):
    # find data for single game 
    match = deliveries[deliveries['id']==match_id]

    #create dictionary holding each overs values
    first_innings_overs, second_innings_overs, first_innings_wkts, second_innings_wkts = get_over_summary(match)    
    
    # sort dictionaries
    first_od = collections.OrderedDict(sorted(first_innings_overs.items()))
    second_od = collections.OrderedDict(sorted(second_innings_overs.items()))
    first_od_wkts = collections.OrderedDict(sorted(first_innings_wkts.items()))
    second_od_wkts = collections.OrderedDict(sorted(second_innings_wkts.items()))
    
    first_match_summary = []
    total = 0
    wks = 0
    for k, v in first_od.items():
        l = []
        l.append(match_id)
        l.append(1)
        l.append(k+1)
        total+=v
        l.append(total)
        wks+=first_od_wkts[k]
        l.append(wks)
        first_match_summary.append(l)
       
    
    first_final_total = total
    second_match_summary = []
    wks = 0
    total = 0
    first_total = 0
    first_wkts = 0 
    for k, v in second_od.items():
        l = []
        l.append(match_id)
        l.append(2)
        l.append(k+1)
        total+=v
        l.append(total)
        wks+=second_od_wkts[k]
        l.append(wks)
        try:
            first_total+=first_od[k]
        except KeyError:
            first_total+=0
        try:
            first_wkts+=first_od_wkts[k]
        except KeyError:
            first_wkts+=0
        l.append(first_total)
        l.append(first_wkts)
        l.append(first_final_total)
        second_match_summary.append(l)
        
        
    #get summary of each game
    #match_summary = get_cumulative_scores(first_od, second_od, first_od_wkts, second_od_wkts, home_bat)
    #match_summary.insert(0, match_id)

    
    return first_match_summary, second_match_summary




In [22]:
def transform_data(ids, df_1, df_2, matches, deliveries):
    for match_id in ids:
        summary_1, summary_2 = get_match_summary(match_id, matches, deliveries)
        for over in summary_1:
            df_1.loc[len(df_1)] = over
        for over in summary_2:
            df_2.loc[len(df_2)] = over

    return df_1, df_2
        

In [23]:
ipl_1, ipl_2 = transform_data(ipl_ids, df_1, df_2, matches, deliveries)

In [24]:
ipl_1

,id,inning,over,runs,wickets
0,335982,1,1,3,0
1,335982,1,2,21,0
2,335982,1,3,27,0
3,335982,1,4,50,0
4,335982,1,5,60,0
...,...,...,...,...,...
15500,1237181,1,16,125,4
15501,1237181,1,17,136,4
15502,1237181,1,18,142,5
15503,1237181,1,19,148,5


In [25]:
ipl_2

,id,inning,over,runs,wickets,first_innings_runs,first_innings_wkts,first_innings_final_total
0,335982,2,1,4,0,3,0,222
1,335982,2,2,9,1,21,0,222
2,335982,2,3,12,2,27,0,222
3,335982,2,4,16,2,50,0,222
4,335982,2,5,24,3,60,0,222
...,...,...,...,...,...,...,...,...
14752,1237181,2,15,126,2,118,4,156
14753,1237181,2,16,137,2,125,4,156
14754,1237181,2,17,147,3,136,4,156
14755,1237181,2,18,154,4,142,5,156


In [26]:
bbl_1, bbl_2 = transform_data(bbl_ids, df_3, df_4, bb_match, bb_ball)

In [27]:
bbl_1

,id,inning,over,runs,wickets
0,524915,1,1,4,0
1,524915,1,2,12,0
2,524915,1,3,13,0
3,524915,1,4,18,0
4,524915,1,5,28,0
...,...,...,...,...,...
7599,1226885,1,16,110,6
7600,1226885,1,17,123,6
7601,1226885,1,18,134,6
7602,1226885,1,19,144,8


In [28]:
bbl_2

,id,inning,over,runs,wickets,first_innings_runs,first_innings_wkts,first_innings_final_total
0,524915,2,1,4,0,4,0,139
1,524915,2,2,10,0,12,0,139
2,524915,2,3,13,0,13,0,139
3,524915,2,4,30,0,18,0,139
4,524915,2,5,39,1,28,0,139
...,...,...,...,...,...,...,...,...
7131,1226885,2,16,124,3,110,6,158
7132,1226885,2,17,138,3,123,6,158
7133,1226885,2,18,147,3,134,6,158
7134,1226885,2,19,158,3,144,8,158


In [29]:
# export all four df's for use in the predictive analysis
ipl_1.to_csv('first_innings.csv')
ipl_2.to_csv('second_innings.csv')
bbl_1.to_csv('bbl_first_innings.csv')
bbl_2.to_csv('bbl_second_innings.csv')

## Final output

The required in-game data has been calculated, including summary stastistics for runs and wickets at the end of each over. This data will be used for in-game predictive analysis